# Using the Python SDK for AutoML with the EON Tuner

<!--- Do not modify the markdown for this example directly! It is generated from a notebook in https://github.com/edgeimpulse/notebooks --->

[![View in Edge Impulse docs](https://raw.githubusercontent.com/edgeimpulse/notebooks/main/.assets/images/ei-badge.svg)](https://docs.edgeimpulse.com/docs/tutorials/ml-and-data-engineering/ei-python-sdk/python-sdk-eon-tuner)
[![Open in Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/edgeimpulse/notebooks/blob/main/notebooks/python-sdk-eon-tuner.ipynb)
[![View on GitHub](https://raw.githubusercontent.com/edgeimpulse/notebooks/main/.assets/images/badge-view-on-github.svg)](https://github.com/edgeimpulse/notebooks/blob/main/notebooks/python-sdk-eon-tuner.ipynb)
[![Download notebook](https://raw.githubusercontent.com/edgeimpulse/notebooks/main/.assets/images/badge-download-notebook.svg)](https://raw.githubusercontent.com/edgeimpulse/notebooks/main/notebooks/python-sdk-eon-tuner.ipynb) 

The [EON Tuner](https://docs.edgeimpulse.com/docs/edge-impulse-studio/eon-tuner) is Edge Impulse's automated machine learning (AutoML) tool to help you find the best combination of blocks and hyperparameters for your model and within your hardware constraints. This example will walk you through uploading data, running the EON Tuner, and interpreting the results.

> **WARNING:** This notebook will add and delete data in your Edge Impulse project, so be careful! We recommend creating a throwaway project when testing this notebook.

To start, create a new project in Edge Impulse. Do not add any data to it.

In [ ]:
# If you have not done so already, install the following dependencies
# !python -m pip install edgeimpulse

In [ ]:
import edgeimpulse as ei
from edgeimpulse.experimental.data import (
    upload_directory
)
from edgeimpulse.experimental.tuner import (
    start_tuner,
    start_custom_tuner,
    check_tuner,
    tuner_report_as_df,
    set_impulse_from_trial,
)

You will need to obtain an API key from an Edge Impulse project. Log into [edgeimpulse.com](https://edgeimpulse.com/) and create a new project. Open the project, navigate to **Dashboard** and click on the **Keys** tab to view your API keys. Double-click on the API key to highlight it, right-click, and select **Copy**.

![Copy API key from Edge Impulse project](https://raw.githubusercontent.com/edgeimpulse/notebooks/main/.assets/images/python-sdk-copy-ei-api-key.png)

Note that you do not actually need to use the project in the Edge Impulse Studio. We just need the API Key.

Paste that API key string in the `ei.API_KEY` value in the following cell:

In [ ]:
# Settings
ei.API_KEY = "ei_dae2..." # Change this to your Edge Impulse API key

In [ ]:
### %%%!!!!!!DELETE ME!!!
ei.API_KEY="ei_001179334ca9bc0458c88cf396499b005182c30dd74c5846f878b7fa481681a2" # replace with your key under dashboard/keys
ei.API_ENDPOINT="http://localhost:4800/v1"
ei.INGESTION_ENDPOINT="http://localhost:4810"

## Upload dataset

We start by downloading the [continuous motion dataset](https://docs.edgeimpulse.com/docs/pre-built-datasets/continuous-gestures) and uploading it to our project.

In [ ]:
# Download and unzip gesture dataset
!mkdir -p dataset/
!wget -P dataset -q https://cdn.edgeimpulse.com/datasets/gestures.zip
!unzip -q dataset/gestures.zip -d dataset/gestures/

In [ ]:
# Upload training dataset
resp = upload_directory(
    directory="dataset/gestures/training",
    category="training",
)
print(f"Uploaded {len(resp.successes)} training samples")

# Upload test dataset
resp = upload_directory(
    directory="dataset/gestures/testing",
    category="testing",
)
print(f"Uploaded {len(resp.successes)} testing samples")

In [ ]:
# Uncomment the following if you want to delete the temporary dataset folder
#!rm -rf dataset/

## Run the Tuner

To start, we need to list the possible target devices we can use for profiling. We need to pick from this list.

In [ ]:
# List the available profile targets
ei.model.list_profile_devices()

You should see a list printed such as:

```
['alif-he',
 'alif-hp',
 'arduino-nano-33-ble',
 'arduino-nicla-vision',
 'portenta-h7',
 'brainchip-akd1000',
 'cortex-m4f-80mhz',
 'cortex-m7-216mhz',
 ...
 'ti-tda4vm']
```

From there, we start the tuner with `start_tuner()` and wait for completion via `check_tuner()`. In this example, we configure the tuner to target for the *alif-hp device*. Since we want to classify the motion, we choose a `classifcation_type` of `classification` and our dataset as motion continous. We constrain our model to a latency of 100ms for running the impulse. 

> **NOTE:** We set the max trials to 3 here. In a real life situation, you will omit this so the tuner decides the best number of trials.

Once the tuner is done, you can print out the results to determine the best combination of blocks and hyperparameters.

In [ ]:
# Start tuner. This will take 30+ minutes.
start_tuner(
    target_device="cortex-m4f-80mhz",
    classification_type="classification",
    dataset_category="motion_continuous",
    target_latency=100,
    tuning_max_trials=1,
)

# Wait while checking the tuner's progress.
state = check_tuner(
    wait_for_completion=True
)

## Print EON Tuner results

The configuration settings and output of the EON Tuner is stored in the variable `state`. You can access the results of the various trials with `state.trials`. Note that some trials can fail, so it's a good idea to test the status of each trial.

From there, you will want to sort the results based on some metric. In this example, we will sort based on *int8* test set accuracy from highest to lowest.

> Note: Edge Impulse supports only one learning block per project at this time (excluding anomaly detection blocks). As a result, we will use the first learning block (e.g. `learning_blocks[0]`) in the list to extract metrics.

In [ ]:
import json

In [ ]:
# Set quantization ("float32" or "int8")
qtzn = "int8"

# Filter out all failed trials
results = [r for r in state.trials if r.status == "completed"]

# Extract float32 accuracies from the trial results
accuracies = []
for result in results:
    accuracy = result.impulse.learn_blocks[0]["metrics"]["test"][qtzn]["accuracy"]
    accuracies.append(accuracy)

# Sort the results based on int8 accuracies
acc_results = zip(accuracies, results)
sorted_results = sorted(acc_results, reverse=True, key=lambda x: list(x)[0])
sorted_results = [result for _, result in sorted_results]

Now that we have the sorted results, we can extract the values we care about. We will print out the following metrics along with the impulse configuration (processing/learning block configuration and hyperparameters) of the top-performing trial.

This will help you determine if the impulse can fit on your target hardware and run fast enough for your needs. The impulse configuration can be used to recreate the processing and learning blocks on Edge Impulse. Later, we will set the project impulse based on the trial ID to simply deploy (rather than re-train).

> **Note:** we assume the first learning block has the metrics we care about.

In [ ]:
### TEST

# The first trial contains the impulse with the top accuracy (for float32)
trial_idx = 0

# Extract the metrics we care about
learn_metadata = sorted_results[trial_idx].impulse.learn_blocks[0]["metadata"]
qtzn_metadata = [m for m in learn_metadata["modelValidationMetrics"] if m.get("type") == qtzn]
tflite_ram = qtzn_metadata[0]["memory"]["tflite"]["ram"]
tflite_rom = qtzn_metadata[0]["memory"]["tflite"]["rom"]
eon_ram = qtzn_metadata[0]["memory"]["eon"]["ram"]
eon_rom = qtzn_metadata[0]["memory"]["eon"]["rom"]

sorted_results[trial_idx]

In [ ]:
# The top performing impulse is the first element
# Note that "top performing" is the highest int8 accuracy on the test set
trial_idx = 0

# Print info about the processing (DSP) blocks and store RAM usage
dsp_ram = 0
print("Processing blocks")
print("===")
for i, dsp_block in enumerate(sorted_results[trial_idx].impulse.dsp_blocks):
    print(f"Processing block {i}")
    print("---")
    print("Block:")
    print(json.dumps(dsp_block["block"], indent=2))
    print("Config:")
    print(json.dumps(dsp_block["config"], indent=2))
    dsp_ram += dsp_block["estimatedPerformance"]["ram"]
print()

# Print info about the learning blocks and store RAM/ROM usage
tflite_ram = 0
tflite_rom = 0
eon_ram = 0
eon_rom = 0
print("Learning blocks")
print("===")
for i, learn_block in enumerate(sorted_results[trial_idx].impulse.learn_blocks):
    print(f"Learn block {i}")
    print("---")
    print("Block:")
    print(json.dumps(learn_block["block"], indent=2))
    print("Config:")
    print(json.dumps(learn_block["config"], indent=2))
    metadata = learn_block["metadata"]
    qtzn_metadata = [m for m in metadata["modelValidationMetrics"] if m.get("type") == qtzn]
    tflite_ram += qtzn_metadata[0]["memory"]["tflite"]["ram"]
    tflite_rom += qtzn_metadata[0]["memory"]["tflite"]["rom"]
    eon_ram += qtzn_metadata[0]["memory"]["eon"]["ram"]
    eon_rom += qtzn_metadata[0]["memory"]["eon"]["rom"]
print()

# Print the metrics
print(f"Metrics ({qtzn})")
print("===")
metrics = sorted_results[trial_idx].impulse.learn_blocks[0]["metrics"]
print(f"Validation accuracy: {metrics['validation'][qtzn]['accuracy']}")
print(f"Test accuracy: {metrics['test'][qtzn]['accuracy']}")
print(f"Estimated processing blocks RAM (bytes): {dsp_ram}")
print(f"Estimated learning blocks RAM (bytes): {tflite_ram}")
print(f"Estimated learning blocks ROM (bytes): {tflite_rom}")
print(f"Estimated learning blocks RAM with EON Compiler (bytes): {eon_ram}")
print(f"Estimated learning blocks ROM with EON Compiler (bytes): {eon_rom}")
print()

### TODO: PRINT LATENCY ESTIMATES!!!

## Graph results

## Results as a DataFrame

If you have [pandas](https://pandas.pydata.org/) installed, you can make the previous section much easier by reporting metrics as a DataFrame.

In [ ]:
# Convert the state metrics into a DataFrame
df = tuner_report_as_df(state)
df.head()

In [ ]:
# Sort the DataFrame by validation (int8) accuracy and print out best trial
df = df.sort_values(by='val_int8_accuracy', ascending=False)
df.iloc[0]

## Configure custom search space

By default, the EON Tuner will make a guess at a search space based on the type of data you uploaded (e.g. using spectral-analysis blocks for feature extraction). As a result, you can run the tuner without needing to construct a search space. However, you may want to define your own search space.

The best way to define a search space is to open your project (after uploading data), head to the **EON Tuner** page, click **Run EON Tuner**, and select the **Space** tab.

![EON Tuner search space](https://raw.githubusercontent.com/edgeimpulse/notebooks/main/.assets/images/python-sdk-eon-tuner-search-space.png)

The search space is defined in JSON format, so we can just copy that to create a dictionary. This is a good place to start for tuning blocks and hyperparameters.

> **Note:** Functions to get available blocks and search space parameters coming soon

In [ ]:
from edgeimpulse_api import (
    OptimizeConfig,
    TunerSpaceImpulse,
)

In [ ]:
# Configure the search space
space = {
    "inputBlocks": [
      {
        "type": "time-series",
        "window": [
          {"windowSizeMs": 9000, "windowIncreaseMs": 9000},
          {"windowSizeMs": 10000, "windowIncreaseMs": 10000}
        ],
        "frequencyHz": [62.5],
        "padZeros": [True]
      }
    ],
    "dspBlocks": [
      {
        "type": "spectral-analysis",
        "analysis-type": ["FFT"],
        "fft-length": [16, 64],
        "scale-axes": [1],
        "filter-type": ["none"],
        "filter-cutoff": [3],
        "filter-order": [6],
        "do-log": [True],
        "do-fft-overlap": [True]
      },
      {
        "type": "spectral-analysis",
        "analysis-type": ["Wavelet"],
        "wavelet": ["haar", "bior1.3"],
        "wavelet-level": [1, 2]
      },
      {"type": "raw", "scale-axes": [1]}
    ],
    "learnBlocks": [
      {
        "id": 4,
        "type": "keras",
        "dimension": ["dense"],
        "denseBaseNeurons": [40, 20],
        "denseLayers": [2, 3],
        "dropout": [0.25, 0.5],
        "learningRate": [0.0005],
        "trainingCycles": [30]
      }
    ]
  }

In [ ]:
# Wrap the search space
ts = TunerSpaceImpulse.from_dict(space)

# Create a custom configuration
config = OptimizeConfig(
    name=None,
    target_device={"name": "cortex-m4f-80mhz"},
    classification_type="classification",
    dataset_category="motion_continuous",
    target_latency=100,
    tuning_max_trials=2,
    space=[ts]
)

In [ ]:
# Start tuner and wait for it to complete
start_custom_tuner(
    config=config
)
state = check_tuner(
    wait_for_completion=True
)

In [ ]:
# Set quantization ("float32" or "int8")
qtzn = "int8"

# Filter out all failed trials
results = [r for r in state.trials if r.status == "completed"]

# Extract float32 accuracies from the trial results
accuracies = []
for result in results:
    accuracy = result.impulse.learn_blocks[0]["metrics"]["test"][qtzn]["accuracy"]
    accuracies.append(accuracy)

# Sort the results based on int8 accuracies
acc_results = zip(accuracies, results)
sorted_results = sorted(acc_results, reverse=True, key=lambda x: list(x)[0])
sorted_results = [result for _, result in sorted_results]

## Set trial as impulse and deploy

We can replace the current impulse with the top performing trial from the EON Tuner. From there, we can deploy it, just like we would any impulse.

In [ ]:
# Get the trial ID and set that to our impulse
trial_id = df.iloc[0].trial_id
set_impulse_from_trial(trial_id)